In [220]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
from bs4 import BeautifulSoup
import requests
import sys
import pickle

import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline
from bs4 import BeautifulSoup, Comment

In [97]:
# url = (f"https://www.baseball-reference.com/leagues/NL/2009-standard-batting.shtml")
# print(f"Checking url: {url}")
# response = requests.get(url)
# page = response.text
    
# # page.replace('!--', '')     
# soup = BeautifulSoup(page,"html5lib")



Checking url: https://www.baseball-reference.com/leagues/NL/2009-standard-batting.shtml


In [82]:

#the following two blocks are for testing purposes to see if the correct data is being brought in

In [130]:
r = requests.get("https://www.baseball-reference.com/leagues/NL/2009-standard-batting.shtml")
soup = BeautifulSoup(r.text, "html.parser")
comments = soup.find_all(string=lambda text:isinstance(text,Comment))

# the element we need has the sentence 'stats_table nav_table' in it
for comment in comments:
    if 'players_standard_batting' in comment:
        table_soup = BeautifulSoup(comment, "html.parser")

table = table_soup.find('table')


In [207]:
table_body = table.find('tbody')

table_rows = table_body.find_all(("tr", {"class" : "full_table non_qual"}) or ("tr", {"class": "full_table"}) or ("tr", {"class": "partial_table non_qual"}))
table_rows
players_stats_one_year_list =[]

for row in table_rows:
    one_player = []
    row_details = row.find_all('td')
#     print(table_details)
    for player_stats in row_details:
        player_info = player_stats.text
        
        one_player.append(player_info)
    players_stats_one_year_list.append(one_player)
players_stats_one_year_list


[['Tony\xa0Abreu#',
  '24',
  'LAD',
  '6',
  '11',
  '8',
  '0',
  '2',
  '0',
  '0',
  '0',
  '1',
  '0',
  '1',
  '3',
  '2',
  '.250',
  '.455',
  '.250',
  '.705',
  '97',
  '2',
  '0',
  '0',
  '0',
  '0',
  '0',
  '/H45'],
 ['Manny\xa0Acosta',
  '28',
  'ATL',
  '34',
  '2',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '.000',
  '.500',
  '.000',
  '.500',
  '48',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1'],
 ['Mike\xa0Adams',
  '30',
  'SDP',
  '34',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1'],
 ['Jeremy\xa0Affeldt*',
  '30',
  'SFG',
  '73',
  '2',
  '2',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '.500',
  '.500',
  '.500',
  '1.000',
  '164',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1'],
 ['Eliezer\xa0Alfonzo',
  '30',
  'SDP',
  '37',
  '117',
  '114',
  '6',
  '20',
  '3',
  '0',
  '2'

In [221]:
def get_soup_url_by_year_league(year):
    #URL
    url = (f"https://www.baseball-reference.com/leagues/NL/{year}-standard-batting.shtml")
    print(f"Checking url: {url}")
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    comments = soup.find_all(string=lambda text:isinstance(text,Comment))

# the element we need has the sentence 'stats_table nav_table' in it
    for comment in comments:
        if 'players_standard_batting' in comment:
            table_soup = BeautifulSoup(comment, "html.parser")

    table = table_soup.find('table')
    
    table_body = table.find('tbody')

    table_rows = table_body.find_all(("tr", {"class" : "full_table non_qual"}) or ("tr", {"class": "full_table"}) or ("tr", {"class": "partial_table non_qual"}))
    table_rows
    players_stats_one_year_list =[]

    for row in table_rows:
        one_player = []
        row_details = row.find_all('td')
#     print(table_details)
        for player_stats in row_details:
            player_info = player_stats.text
        
            one_player.append(player_info)
        players_stats_one_year_list.append(one_player)

    #Beautiful Soup for web scraping - table is now pure HTML (initially SHTML)
  
        
    
   
    
    
    col_names_list=[]

    col_name_all = table.find_all('tr', {'class':'thead'})
    col_name_final = col_name_all[0]

    for name in col_name_final.find_all("th"):
        col_names_list.append(name.text)
    col_names_list_final = col_names_list[1:]
        
        
    
       
    df = pd.DataFrame(data=players_stats_one_year_list, columns = col_names_list_final)
    
    return df


In [222]:
get_soup_url_by_year_league(2000)

Checking url: https://www.baseball-reference.com/leagues/NL/2000-standard-batting.shtml


,Name,Age,Tm,G,PA,AB,R,H,2B,3B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary
0,Kurt Abbott,31,NYM,79,173,157,22,34,7,1,...,.389,.672,71,61,2,1,0,1,2,6H4/58
1,Bobby Abreu*,26,PHI,154,680,576,103,182,42,10,...,.554,.970,143,319,12,1,0,3,9,*9/H
2,Juan Acevedo,30,MIL,60,2,1,1,0,0,0,...,.000,.500,44,0,0,0,0,0,0,1
3,Terry Adams,27,LAD,63,3,2,0,0,0,0,...,.000,.000,-100,0,0,0,1,0,0,1
4,Benny Agbayani,28,NYM,119,415,350,59,101,19,1,...,.477,.868,122,167,6,7,0,3,2,*7H9/8D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,Todd Zeile,34,NYM,153,623,544,67,146,36,3,...,.467,.823,110,254,15,2,0,3,4,*3/H
815,Chad Zerbe*,28,SFG,4,0,0,0,0,0,0,...,,,,0,0,0,0,0,0,/1
816,Eddie Zosky,32,HOU,4,4,4,0,0,0,0,...,.000,.000,-100,0,0,0,0,0,0,/H
817,Julio Zuleta,25,CHC,30,73,68,13,20,8,0,...,.544,.887,122,37,2,3,0,0,0,3H/7


In [223]:
moneyball_master_df = pd.DataFrame()
for inp_year in range(1999,2020,1): 
    #Empty df to start
    
    
    
    df_1 = get_soup_url_by_year_league(inp_year)
    
    
    moneyball_master_df = pd.concat([moneyball_master_df,df_1])
    print(f"{inp_year} has been added to master")
    print("**********************************")

Checking url: https://www.baseball-reference.com/leagues/NL/1999-standard-batting.shtml
1999 has been added to master
**********************************
Checking url: https://www.baseball-reference.com/leagues/NL/2000-standard-batting.shtml
2000 has been added to master
**********************************
Checking url: https://www.baseball-reference.com/leagues/NL/2001-standard-batting.shtml
2001 has been added to master
**********************************
Checking url: https://www.baseball-reference.com/leagues/NL/2002-standard-batting.shtml
2002 has been added to master
**********************************
Checking url: https://www.baseball-reference.com/leagues/NL/2003-standard-batting.shtml
2003 has been added to master
**********************************
Checking url: https://www.baseball-reference.com/leagues/NL/2004-standard-batting.shtml
2004 has been added to master
**********************************
Checking url: https://www.baseball-reference.com/leagues/NL/2005-standard-batting.

In [224]:
moneyball_master_df

,Name,Age,Tm,G,PA,AB,R,H,2B,3B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary
0,Jim Abbott*,31,MIL,18,24,21,0,2,0,0,...,.095,.190,-51,2,0,0,3,0,0,1
1,Kurt Abbott,30,COL,96,305,286,41,78,17,2,...,.430,.740,68,123,4,0,2,1,0,4H/3689
2,Bobby Abreu*,25,PHI,152,662,546,118,183,35,11,...,.549,.995,147,300,13,3,0,4,8,*9/DH
3,Juan Acevedo,29,STL,49,22,20,0,1,0,0,...,.050,.100,-75,1,0,0,2,0,0,1
4,Terry Adams,26,CHC,49,2,2,0,0,0,0,...,.000,.000,-100,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,Daniel Zamora*,26,NYM,17,0,0,0,0,0,0,...,,,,0,0,0,0,0,0,1
829,Ryan Zimmerman,34,WSN,52,190,171,20,44,9,0,...,.415,.736,88,71,4,0,0,2,0,3/HD
830,Ben Zobrist#,38,CHC,47,176,150,24,39,5,0,...,.313,.671,79,47,6,1,0,2,0,49/7HD16
831,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [225]:

moneyball_master_df.columns = df.columns.map(str.strip)
moneyball_master_df.columns
import pickle
with open('moneyball_master_df.pkl', 'wb') as picklefile:
    pickle.dump(moneyball_master_df, picklefile)